# AI-Powered Scenario Optimization: Warm-up

## Introduction


In this hackathon, you'll be developing an AI-powered solution to optimize manufacturing processes for sustainability. Specifically, you'll be working with HappyBikes, a company that produces traditional and electronic bicycles, to optimize their manufacturing processes for cost and environmental impact.

This notebook will introduce you to the key optimization techniques.

It covers:
1. The optimization problem overview
2. Introduction to Genetic Algorithms
3. Introduction to Particle Swarm Optimization
4. How to apply these techniques to the bicycle manufacturing problem
5. Simple examples to illustrate the concepts

## 1. Optimization Problem Overview

### The Challenge

HappyBikes produces traditional and electronic bicycles, each composed of multiple components (saddles, wheels, brakes, etc.). Each component can be manufactured using different materials and production methods ("recipes").

Different "recipes" lead to varying:
- Production costs
- CO₂ emissions
- Water consumption
- Product durability

### Optimization Goals

- **Primary Objectives**: Minimize costs AND minimize CO₂ emissions
- **Secondary Objectives**: When primary metrics are tied, optimize for:
  - Lower water consumption
  - Higher product durability (calculated as average of materials durability)

### Constraints

The optimization is subject to various constraints, such as:
- Material-based constraints (e.g., maximum 5kg of copper)
- Parameter-based constraints (e.g., minimum durability of 5 months)
- Cost constraints (e.g., maximum cost of $100)

This is a **multi-objective optimization problem** with **constraints**, which makes it challenging to solve using traditional optimization methods. This is where AI-powered optimization techniques come in!

## 2. Introduction to Genetic Algorithms

### What are Genetic Algorithms?

Genetic Algorithms (GAs) are optimization algorithms inspired by the process of natural selection. They work by evolving a population of candidate solutions over several generations with the aim of improving their fitness to solve a specific problem. They are particularly well-suited for complex optimization problems where the search space is large and potentially discontinuous.

### Key Concepts

1. **Chromosome/Individual**: A potential solution to the problem, represented as a string of genes.
2. **Population**: A collection of individuals (potential solutions).
3. **Fitness Function**: A function that evaluates how good a solution is.
4. **Selection**: The process of choosing individuals for reproduction based on their fitness.
5. **Crossover**: The process of combining genetic material from two parents to create offspring.
6. **Mutation**: Random changes to individuals to maintain genetic diversity.

### The Genetic Algorithm Process

1. **Initialization**: Create an initial population of (random) solutions.
2. **Evaluation**: Calculate the fitness of each individual in the population.
3. **Selection**: Select individuals for reproduction based on their fitness.
4. **Crossover**: Create new individuals by combining genetic material from selected parents.
5. **Mutation**: Introduce random changes to some individuals to maintain genetic diversity.
6. **Replacement**: Replace the old population with the new population.
7. **Termination**: Repeat steps 2-6 until a termination condition is met (e.g., maximum number of generations, satisfactory solution found).

### Simple Example of a Genetic Algorithm

Let's implement a simple genetic algorithm to find the maximum value of the function $f(x) = x^2$ in the range $[0, 31]$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random


# Define the fitness function
def fitness_function(x):
    return x**2


# Convert binary string to integer
def binary_to_int(binary_string):
    return int(binary_string, 2)


# Generate a random individual (binary string of length 5)
def generate_individual():
    return "".join(random.choice("01") for _ in range(5))


# Generate initial population
def generate_population(size):
    return [generate_individual() for _ in range(size)]


# Select individual with the highest fitness score for reproduction using tournament selection
def selection(population, fitnesses, k=3):
    # Select k random individuals from the population
    selected_indices = random.sample(range(len(population)), k)
    # Evaluate their fitness
    selected_fitnesses = [fitnesses[i] for i in selected_indices]
    # Select the individual with the highest fitness
    return population[selected_indices[np.argmax(selected_fitnesses)]]


# Perform single-point crossover
def crossover(parent1, parent2):
    # Choose a random crossover point
    crossover_point = random.randint(1, len(parent1) - 1)
    # Create offspring by combining parts of parents
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2


# Perform mutation
def mutation(individual, mutation_rate=0.1):
    # Flip each bit with probability mutation_rate
    mutated = ""
    for bit in individual:
        if random.random() < mutation_rate:
            mutated += "1" if bit == "0" else "0"
        else:
            mutated += bit
    return mutated


# Run the genetic algorithm
def genetic_algorithm(population_size=5, generations=10):
    # Generate initial population
    population = generate_population(population_size)

    # Keep track of the best solution found so far
    best_individual = None
    best_fitness = -float("inf")

    # Keep track of the average fitness per generation
    avg_fitness_history = []
    best_fitness_history = []

    # Run for a fixed number of generations
    for generation in range(generations):
        # Evaluate fitness of each individual
        fitnesses = np.array(
            [fitness_function(binary_to_int(ind)) for ind in population]
        )

        # Update best solution
        max_fitness_idx = np.argmax(fitnesses)
        if fitnesses[max_fitness_idx] > best_fitness:
            best_individual = population[max_fitness_idx]
            best_fitness = fitnesses[max_fitness_idx]

        # Record statistics
        avg_fitness_history.append(np.mean(fitnesses))
        best_fitness_history.append(best_fitness)

        # Create new population
        new_population = []

        # Elitism: keep the best individual
        new_population.append(population[max_fitness_idx])

        # Create the rest of the new population
        while len(new_population) < population_size:
            # Select parents
            parent1 = selection(np.array(population), fitnesses)
            parent2 = selection(np.array(population), fitnesses)

            # Perform crossover
            child1, child2 = crossover(parent1, parent2)

            # Perform mutation
            child1 = mutation(child1)
            child2 = mutation(child2)

            # Add children to new population
            new_population.append(child1)
            if len(new_population) < population_size:
                new_population.append(child2)

        # Replace old population with new population
        population = new_population

        print(f"Generation {generation + 1}:")
        print(f"Best individual: {best_individual}")
        print(f"Best individual (decimal): {binary_to_int(best_individual)}")
        print(f"Best fitness: {best_fitness}")
        print("--------------------------------------------------")

    return best_individual, best_fitness, avg_fitness_history, best_fitness_history

In [ ]:
# Run the genetic algorithm
best_individual, best_fitness, avg_fitness_history, best_fitness_history = (
    genetic_algorithm()
)

# Plot fitness history
plt.figure(figsize=(10, 5))
plt.plot(avg_fitness_history, label="Average Fitness")
plt.plot(best_fitness_history, label="Best Fitness")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.title("Fitness History")
plt.legend()
plt.grid(True)
plt.show()

If you launch above cell multiple times you will see that the process is not deterministic and not always the best solution is found. 

However, always the GA tries to maximize the best solution and goes in correct direction.


## 3. Introduction to Particle Swarm Optimization

### What is Particle Swarm Optimization?

Particle Swarm Optimization (PSO) is another population-based optimization algorithm, inspired by the social behavior of birds flocking or fish schooling. Unlike genetic algorithms, PSO doesn't use evolutionary operators like crossover and mutation. Instead, it uses the concept of "particles" moving through the search space, guided by their own best known position and the best known position of the entire swarm.

### Key Concepts

1. **Particle**: A potential solution to the problem, represented as a position in the search space.
2. **Velocity**: The rate at which a particle changes its position.
3. **Personal Best (pbest)**: The best position a particle has found so far.
4. **Global Best (gbest)**: The best position found by any particle in the swarm.
5. **Inertia Weight**: Controls the impact of the previous velocity on the current velocity.
6. **Cognitive Component**: Controls the influence of the particle's personal best position.
7. **Social Component**: Controls the influence of the global best position.

### The PSO Process

1. **Initialization**: Create a swarm of particles with random positions and velocities.
2. **Evaluation**: Calculate the fitness of each particle.
3. **Update Personal Best**: If a particle's current position is better than its personal best, update its personal best.
4. **Update Global Best**: If a particle's current position is better than the global best, update the global best.
5. **Update Velocity and Position**: Update each particle's velocity and position based on its previous velocity, personal best, and global best.
6. **Termination**: Repeat steps 2-5 until a termination condition is met.

In [ ]:
# Define a simple objective function to minimize: f(x,y) = x^2 + y^2
# This is a simple bowl-shaped function with minimum at (0,0)
def objective_function(position):
    """
    Our objective function that we want to minimize.
    In this case, it's a simple bowl-shaped function.
    """
    x, y = position  # Unpack the position array to x and y values
    return x**2 + y**2  # Calculate x^2 + y^2

In [ ]:
class Particle:
    """
    A single particle in the swarm.
    Each particle represents a potential solution to our problem.
    """

    def __init__(self, bounds):
        """
        Initialize a new particle with random position and velocity.

        Args:
            bounds: List of tuples defining the search space [(x_min, x_max), (y_min, y_max)]
        """
        # Initialize random position within the specified bounds
        self.position = np.array(
            [
                np.random.uniform(bounds[0][0], bounds[0][1]),
                np.random.uniform(bounds[1][0], bounds[1][1]),
            ]
        )

        # Initialize random velocity
        self.velocity = np.random.uniform(
            -0.5, 0.5, 2
        )  # 2-dimensional velocity for x and y from range -0.5 to 0.5

        # Initialize personal best position to current position
        self.best_position = self.position.copy()

        # Initialize best score (function value) to infinity (since we're minimizing)
        self.best_score = float("inf")

    def update_personal_best(self):
        """Update the particle's personal best position if current position is better."""
        # Calculate current score (function value) at current position
        current_score = objective_function(self.position)

        # If current score is better than best score ever found by this particle
        if current_score < self.best_score:
            # Update personal best position and score
            self.best_position = self.position.copy()
            self.best_score = current_score

    def update_velocity(self, global_best_position, w=0.5, c1=1.5, c2=1.5):
        """
        Update the particle's velocity based on inertia, cognitive component, and social component.

        Args:
            global_best_position: Best position found by any particle in the swarm
            w: Inertia weight - controls impact of previous velocity
            c1: Cognitive weight - controls particle's tendency to return to personal best
            c2: Social weight - controls particle's tendency to move toward swarm's best
        """
        # Random factors for cognitive and social components
        r1 = np.random.random(2)
        r2 = np.random.random(2)

        # Inertia component - tendency to continue moving in same direction
        inertia = w * self.velocity

        # Cognitive component - attraction to particle's personal best position
        cognitive_component = c1 * r1 * (self.best_position - self.position)

        # Social component - attraction to swarm's global best position
        social_component = c2 * r2 * (global_best_position - self.position)

        # Update velocity as sum of these components
        self.velocity = inertia + cognitive_component + social_component

    def update_position(self, bounds):
        """
        Update the particle's position based on its velocity and ensure it stays within bounds.

        Args:
            bounds: List of tuples defining the search space [(x_min, x_max), (y_min, y_max)]
        """
        # Update position by adding velocity
        self.position = self.position + self.velocity

        # Ensure position stays within bounds
        for i in range(len(bounds)):
            if self.position[i] < bounds[i][0]:
                self.position[i] = bounds[i][0]
            elif self.position[i] > bounds[i][1]:
                self.position[i] = bounds[i][1]


In [ ]:
def particle_swarm_optimization(
    num_particles=20, max_iterations=100, bounds=[(-10, 10), (-10, 10)]
):
    """
    Main PSO algorithm.

    Args:
        num_particles: Number of particles in the swarm
        max_iterations: Maximum number of iterations
        bounds: Search space bounds for each dimension

    Returns:
        global_best_position: The best solution found
        global_best_score: The score (function value) of the best solution
        position_history: History of global best positions for visualization
    """
    # Initialize a swarm of particles
    swarm = [Particle(bounds) for _ in range(num_particles)]

    # Initialize global best position and score
    global_best_position = None
    global_best_score = float("inf")

    # Create a list to store history of global best positions (for visualization)
    position_history = []

    # Main PSO loop
    for iteration in range(max_iterations):
        # For each particle in the swarm
        for particle in swarm:
            # Calculate current score (function value) at particle's position
            current_score = objective_function(particle.position)

            # Update global best if this particle's position is better
            if current_score < global_best_score:
                global_best_position = particle.position.copy()
                global_best_score = current_score

                # Add to history for visualization
                position_history.append(
                    (global_best_position.copy(), global_best_score)
                )

            # Update particle's personal best
            particle.update_personal_best()

        # After evaluating all particles, update their velocities and positions
        for particle in swarm:
            particle.update_velocity(global_best_position)
            particle.update_position(bounds)

        # Print current best solution (every 10 iterations)
        if iteration % 10 == 0:
            print(f"Iteration {iteration}: Best Score = {global_best_score}")
            print(f"Best Position = {global_best_position}")

    # Print final results
    print("\nPSO Optimization Complete!")
    print(f"Best Solution: {global_best_position}")
    print(f"Best Score: {global_best_score}")

    return global_best_position, global_best_score, position_history


def visualize_result(position_history, bounds):
    """
    Visualize the optimization process and the objective function.

    Args:
        position_history: History of global best positions during optimization
        bounds: Search space bounds
    """
    # Create a grid of points to evaluate the objective function
    x = np.linspace(bounds[0][0], bounds[0][1], 100)
    y = np.linspace(bounds[1][0], bounds[1][1], 100)
    X, Y = np.meshgrid(x, y)
    Z = np.zeros_like(X)

    # Calculate function values for all grid points
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i, j] = objective_function([X[i, j], Y[i, j]])

    # Create contour plot of the objective function
    plt.figure(figsize=(10, 8))
    plt.contourf(X, Y, Z, 50, cmap="viridis", alpha=0.8)
    plt.colorbar(label="Objective Function Value")

    # Extract positions and scores from history
    positions = np.array([p[0] for p in position_history])
    scores = np.array([p[1] for p in position_history])

    # Plot the particle's path
    plt.plot(
        positions[:, 0],
        positions[:, 1],
        "o-",
        color="red",
        markersize=3,
        linewidth=1,
        label="Optimization Path",
    )

    # Highlight the final best position
    plt.scatter(
        positions[-1, 0],
        positions[-1, 1],
        color="white",
        s=100,
        edgecolor="black",
        label="Best Solution",
    )

    plt.title("Particle Swarm Optimization")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.7)

    plt.show()


In [ ]:
# Define search space bounds: x and y are both in range [-10, 10]
bounds = [(-10, 10), (-10, 10)]

# Run PSO algorithm
best_position, best_score, position_history = particle_swarm_optimization(
    num_particles=50,  # Number of particles in the swarm
    max_iterations=100,  # Number of iterations to run
    bounds=bounds,  # Search space bounds
)

# Visualize the results
visualize_result(position_history, bounds)

## 4. Applying Optimization Techniques to the Bicycle Manufacturing Problem

Now that we understand the basics of Genetic Algorithms and Particle Swarm Optimization, let's discuss how these techniques can be applied to the bicycle manufacturing optimization problem.

### Problem Representation

First, we need to decide how to represent a solution to the problem. In this case, a solution is a specific configuration of components for a bicycle. We can represent this as a vector where each element corresponds to a specific component choice.

For example, if we have 5 different types of components (saddle, wheels, brakes, frame, handlebars), and each component has multiple versions (e.g., saddle_A, saddle_B, etc.), we can represent a solution as a vector of indices, where each index corresponds to the chosen version for each component type.

### Fitness Function / Objective Function

The fitness function (for GA) or objective function (for PSO) should evaluate how good a solution is based on the optimization goals. In this case, we have multiple objectives:

1. Minimize production costs
2. Minimize CO₂ emissions
3. Minimize water consumption (secondary)
4. Maximize average product durability (secondary)

We can use a weighted sum approach to combine these objectives into a 2-step fitness value: 
- primary: minimize sum of coats and CO₂ emissions
- secondary: miminize sum of all metrics (that requires change of sign of product durability, as this is the only metric we want to maximize).


### Handling Constraints

Both GA and PSO need to handle constraints such as:
- Material-based constraints (e.g., maximum 5kg of copper)
- Parameter-based constraints (e.g., minimum durability of 5 months)

There are several ways to handle constraints:
1. **Penalty Functions**: Add a penalty term to the fitness function for solutions that violate constraints.
2. **Repair Mechanisms**: Modify invalid solutions to make them valid.
3. **Constrained Optimization**: Use specialized algorithms that directly handle constraints.

### Genetic Algorithm Approach

For a GA approach to the bicycle manufacturing problem:

1. **Representation**: Each individual (chromosome) represents a specific configuration of components for a bicycle.
2. **Initialization**: Generate a random population of bicycle configurations.
3. **Fitness Evaluation**: Calculate the fitness of each configuration based on cost, CO₂ emissions, water consumption, and durability.
4. **Selection**: Select configurations for reproduction based on their fitness.
5. **Crossover**: Create new configurations by combining components from selected parents.
6. **Mutation**: Introduce random changes to some configurations to maintain diversity.
7. **Replacement**: Replace the old population with the new population.
8. **Termination**: Repeat steps 3-7 until a termination condition is met.

### Particle Swarm Optimization Approach

For a PSO approach to the bicycle manufacturing problem:

1. **Representation**: Each particle represents a specific configuration of components for a bicycle.
2. **Initialization**: Generate a random swarm of bicycle configurations.
3. **Fitness Evaluation**: Calculate the fitness of each configuration based on cost, CO₂ emissions, water consumption, and durability.
4. **Update Personal Best**: If a configuration's current fitness is better than its personal best, update its personal best.
5. **Update Global Best**: If a configuration's current fitness is better than the global best, update the global best.
6. **Update Velocity and Position**: Update each configuration's velocity and position based on its previous velocity, personal best, and global best.
7. **Termination**: Repeat steps 3-6 until a termination condition is met.

## 5. Conclusion

In this warm-up notebook, we introduced the optimization problem for the bicycle manufacturing challenge and provided an overview of Genetic Algorithms and Particle Swarm Optimization. We also provided simple examples to illustrate these concepts.

In the next steps, you'll be able to apply these techniques to the bicycle manufacturing problem using the provided dataset and optimization framework. Good luck, and happy optimizing!